In [24]:
import tensorflow as tf
import numpy as np
import pickle
x = 4
print(x)

4


In [25]:
def generator(data):
    """Generates data for the dataset."""
    for (gr_1, gr_2, y) in data:
        yield (
            tuple(tf.convert_to_tensor(arr, dtype=tf.float32) for arr in gr_1),
            tuple(tf.convert_to_tensor(arr, dtype=tf.float32) for arr in gr_2),
            y
        )

def dataset_tf(data):
    """Creates a TensorFlow dataset from the data."""
    # Define the output types and shapes
    output_signature = (
        (tf.TensorSpec(shape=None, dtype=tf.float32),
         tf.TensorSpec(shape=None, dtype=tf.float32),
         tf.TensorSpec(shape=None, dtype=tf.float32)),
        (tf.TensorSpec(shape=None, dtype=tf.float32),
         tf.TensorSpec(shape=None, dtype=tf.float32),
         tf.TensorSpec(shape=None, dtype=tf.float32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )

    # Create a dataset
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(data), # Using a lambda function to call generator with data
        output_signature=(output_signature[0], output_signature[1], output_signature[2])
    )

    return dataset


#FIXME: This function is not working properly; everything above is working.
def dataset_prep(dataset, val_split=0.1, test_split=0.1, batch_size=32):
    # Shuffle the entire dataset
    BUFFER_SIZE = 10000
    dataset = dataset.shuffle(BUFFER_SIZE)

    # Calculate the sizes for splitting
    total_size = sum(1 for _ in dataset)
    val_size = int(val_split * total_size)
    test_size = int(test_split * total_size)
    train_size = total_size - val_size - test_size

    # Split into training, validation, and test sets
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size + val_size)

    # Batch and prefetch for all datasets
    train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, val_dataset, test_dataset



In [20]:
# Test on dummy data 
data = [
    [[np.array([[0.5645, 0.2412], [0.9563, 0.1425]]), np.array([0.8741, 0.3524, 0.4567]), np.array([[0.6324, 0.8632], [0.7841, 0.5124]])],
     [np.array([[0.7234, 0.4234], [0.7432, 0.9876]]), np.array([0.1234, 0.5310, 0.9865]), np.array([[0.6345, 0.2354], [0.7865, 0.5432]])], 0],
    [[np.array([[0.4523, 0.9876], [0.2413, 0.1532]]), np.array([0.3423, 0.6543, 0.7654]), np.array([[0.5432, 0.7654], [0.8675, 0.2134]])],
     [np.array([[0.1234, 0.9765], [0.7654, 0.2345]]), np.array([0.8765, 0.2345, 0.5432]), np.array([[0.1234, 0.4567], [0.7654, 0.9876]])], 0],
    [[np.array([[0.7654, 0.1234], [0.4567, 0.6543]]), np.array([0.9765, 0.1234, 0.8765]), np.array([[0.2345, 0.5678], [0.1234, 0.7654]])],
     [np.array([[0.5432, 0.7654], [0.8765, 0.4321]]), np.array([0.1234, 0.5678, 0.4321]), np.array([[0.7654, 0.1234], [0.2345, 0.6543]])], 0],
    [[np.array([[0.2345, 0.6789], [0.1234, 0.2345]]), np.array([0.8765, 0.1234, 0.4321]), np.array([[0.5432, 0.2345], [0.7654, 0.1234]])],
     [np.array([[0.9876, 0.1234], [0.4321, 0.5678]]), np.array([0.6543, 0.1234, 0.7654]), np.array([[0.8765, 0.2345], [0.1234, 0.4321]])], 0],
    [[np.array([0.5432, 0.8765, 0.1234]), np.array([0.7654, 0.2345]), np.array([0.4321, 0.5678, 0.8765])],
     [np.array([0.1234, 0.6789]), np.array([0.7654, 0.1234, 0.4567]), np.array([0.2345, 0.5678, 0.1234])], 1]
]

# Test for generator() and dataset_tf()
my_dataset = dataset_tf(data)
train_dataset, val_dataset, test_dataset = dataset_prep(my_dataset, val_split=0.1, test_split=0.1, batch_size=32)
# Iterate through the dataset
for item in train_dataset:
    graph1, graph2, label = item
    print('Graph 1:', graph1)
    print('Graph 2:', graph2)
    print('Label:', label.numpy())

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_7_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [2,2] and element 2 had shape [3]. [Op:IteratorGetNext]

In [26]:
# Testing real data
pseudodata_path = r"C:\Users\xmoot\Desktop\Data\ssl-seizure-detection\patient_pseudolabeled\relative_positioning\jh101_12s_7min_np_2.pkl"

# # Load data
data = pickle.load(open(pseudodata_path, "rb"))

dataset = dataset_tf(data)

dataset = dataset.shuffle(1000).batch(32).prefetch(10)
# train_loader, val_loader, test_loader = dataloader_tf(data, batch_size=32, val_size=0.2, test_size=0.1, seed=16)
for item in dataset:
    graph1, graph2, label = item
    print('Graph 1 Adjaceny matrix:', graph1[0])
    print('Graph 1 Node features:', graph1[1])
    print('Graph 1 Edge features:', graph1[2])
    print('Graph 2 Adjaceny matrix:', graph2[0])
    print('Graph 2 Node features:', graph2[1])
    print('Graph 2 Edge features:', graph2[2])
    print('Label:', label.numpy())
    break


Graph 1 Adjaceny matrix: tf.Tensor(
[[[ 2.441092    0.6606946  -0.12578358 ...  0.83818007 -1.5898882
   -0.8057713 ]
  [ 0.6606946   2.441092    1.6717803  ...  0.6053486  -2.1600127
   -1.9578215 ]
  [-0.12578358  1.6717803   2.441092   ...  0.4384336  -1.4936295
   -1.5495999 ]
  ...
  [ 0.83818007  0.6053486   0.4384336  ...  2.441092   -1.4207016
   -1.2727665 ]
  [-1.5898882  -2.1600127  -1.4936295  ... -1.4207016   2.441092
    1.8921069 ]
  [-0.8057713  -1.9578215  -1.5495999  ... -1.2727665   1.8921069
    2.441092  ]]

 [[ 2.441092    0.6606946  -0.12578358 ...  0.83818007 -1.5898882
   -0.8057713 ]
  [ 0.6606946   2.441092    1.6717803  ...  0.6053486  -2.1600127
   -1.9578215 ]
  [-0.12578358  1.6717803   2.441092   ...  0.4384336  -1.4936295
   -1.5495999 ]
  ...
  [ 0.83818007  0.6053486   0.4384336  ...  2.441092   -1.4207016
   -1.2727665 ]
  [-1.5898882  -2.1600127  -1.4936295  ... -1.4207016   2.441092
    1.8921069 ]
  [-0.8057713  -1.9578215  -1.5495999  ... -1.2727

In [27]:
import os
import pickle

from tensorflow.keras.optimizers import Adam
from model import relative_positioning
from dataloader_tf import dataloader_tf, dataset_tf, generator
from evaluation import f1_score, training_curves, eval

model = relative_positioning(fltrs_out=64, l2_reg=1e-3)
    
# Optimization algorithm
optimizer = Adam(lr=1e-3)

# Compile model
model.compile(optimizer=optimizer, loss="binary_crossentropy")

# Num epochs
epochs = 10

for epoch in range(epochs):
    for batch, (inputs, targets) in enumerate(dataset):
        loss_value = model.train_on_batch(inputs, targets)
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Loss: {loss_value}")
model.save('testmodel.h5')

Start of train_loader:
<_BatchDataset element_spec=((TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)), (TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


: 

: 